In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import torch
from dataset import Model10NetDataset
from model import TNet

In [14]:
torch.cuda.is_available()

True

In [15]:
dataset = Model10NetDataset(path="./ModelNet10")

In [16]:
point_cloud = dataset[0][0]
point_cloud.shape

torch.Size([1601, 3])

In [17]:
point_cloud = point_cloud.unsqueeze(0)
point_cloud.shape
point_cloud = point_cloud.to("cuda")
point_cloud.shape

torch.Size([1, 1601, 3])

In [ ]:
def stack(x):
    # Add a dimension to make it (1, N, C)
    x_unsqueezed = x.unsqueeze(0)

    # Concatenate along the new dimension to make it (2, N, C)
    x_stacked = torch.cat([x_unsqueezed, x_unsqueezed], dim=0)

    return x_stacked

In [54]:
tnet = TNet(in_features=3)
tnet = tnet.to("cuda")
tnet

TNet(
  (linear1): Linear(in_features=3, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=1024, bias=True)
  (relu): ReLU()
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Linear(in_features=256, out_features=9, bias=True)
  )
)

In [55]:
out = tnet(point_cloud)
out.shape

torch.Size([1, 1024])


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])

In [53]:
out

tensor([[-0.1096,  0.3557,  0.4177,  0.0255,  0.2147, -0.0453, -0.6353, -0.0045,
          0.4330]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [57]:
import torch.nn as nn

In [61]:
nn.BatchNorm1d(10)(torch.rand((1, 10)))

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 10])